* Generic Algorithms
* Particle Swarm Optimation

In [ ]:
# Preprocessing

# Removing missing values
df = csvFile.copy()
df.dropna(inplace = True)

# Removing duplicates
df.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)

# Removing anomalies
df = df.drop(df[df.ph < 4].index, inplace=True)

print(csvFile.size)
print(df.size)

In [ ]:
# Cross folding
# folding for 3,4,5
for curr in range(3,6):
    num_folds = curr
    score = 0
    # implement cross validation
    # https://de.mathworks.com/discovery/cross-validation.html
    for i in range(0, num_folds):
        partition_size = int(np.floor(csvFile.shape[0] / num_folds))
        left_border = partition_size * i
        right_border = partition_size * (i + 1)
        testInds = numsMe[left_border:right_border]
        trainInds = np.concatenate((numsMe[0:left_border], numsMe[right_border:csvFile.shape[0]]))

        TrainX, TestX = X[trainInds], X[testInds]
        TrainY, TestY = Y[trainInds], Y[testInds]

        clf_dt = tree.DecisionTreeClassifier()
        clf_dt = clf_dt.fit(TrainX, TrainY)

        predY_dt = clf_dt.predict(TestX)

        score = score + accuracy_score(TestY, predY_dt)
    plt.bar(num_folds, score/num_folds)
plt.show()

In [ ]:
# One hot encoding
Y_Arr = np.zeros((Y.shape[0], 3))
for i in range(Y.shape[0]):
    if Y[i] == 1:
        Y_Arr[i] = [1, 0, 0]
    elif Y[i] == 2:
        Y_Arr[i] = [0, 1, 0]
    elif Y[i] == 3:
        Y_Arr[i] = [0, 0, 1]

# Accuracy
correct_preds = 0
for index, i in enumerate(preds):
    if np.array_equal(i, TestY[index]):
        correct_preds = correct_preds + 1
accuracy = correct_preds / len(TestY)
print("Accuracy=" + str(accuracy))

In [ ]:
# Normalization
for i in range(X.shape[1]):
    # Scaling normalization
    X[:, i] = (X[:, i] - np.amin(X[:, i])) / (np.amax(X[:, i]) - np.amin(X[:, i]))
